## 数据库

In [1]:
import sys
import re
from pyhive import hive
import pandas as pd
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import scipy.stats as stats
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler
import requests
import json
import time
from matplotlib import pyplot as plt

from matplotlib.font_manager import FontProperties
font = FontProperties('simhei', size=20) #中文字体


pd.set_option('display.max_columns', None) #显示所有列
sys.path.append('/data/datamining/Py_functions/get_data')
from get_data_func import * ##取数function  


In [18]:
raw_data = pd.read_excel('pcr-2020-2022.xlsx', sheet_name = '抗原结果明细表')

# 统计
统计在爆发日期多长时间前开始检出PED，以及在爆发时间后多长时间内停止检出ped

In [13]:
data_33w = pd.read_csv('33M_data.csv')
data_7_12m = pd.read_csv('21_07m_12m_data.csv') # 7月到12月 
data_11_3m = pd.read_csv('20_11m_03m_data.csv') # 11-3月

# 新模型
正样本：爆发前（*a*）天需要推送警告的为正样本，即1-3号和15-17号
负样本：警告前，清零后的样本，即26-30号和14号
输出：该场区该天是否需要推送警告

### 贴标签

In [20]:
#利润损失表
benifit_raw_11_3m  = pd.read_excel('利润损失分析 11-3月.xlsx')
benifit_raw_7_12m = pd.read_excel('利润损失分析 7-12.xlsx')
benifit_raw_year = pd.read_excel('利润损失分析全年.xlsx')

#去重重复记录的场区（重复是因为记录的不同的单元）
benifit_list_11_3m = benifit_raw_11_3m.loc[:, ['分场', '业务日期']].drop_duplicates()
benifit_list_7_12m = benifit_raw_7_12m.loc[:, ['分场', '业务日期']].drop_duplicates()
benifit_list_year = benifit_raw_year.loc[:,['分场', '业务日期']].drop_duplicates()

例子：卧龙牧原1场3区生长场

In [497]:
# 首先确定10月31号之间没有记录。因为如果有，说明发病更早，有可能是上一轮没有被清零。所以这里可以被贴标签的场区和世界
def good_to_research(fieldname, starttime):
    '''判断该场区是否值得被研究。10月31号之前没有记录才能被研究，并且之后有数据'''
    part_data = benifit_list_year.loc[(benifit_list_year.业务日期 < starttime) & (benifit_list_year.分场  == fieldname)] #31号前没数据
    part2_data = benifit_list_year.loc[(benifit_list_year.业务日期 >= starttime) & (benifit_list_year.分场  == fieldname)] #31号后有数据
    return (part_data.empty & (not part2_data.empty) )#输出是否值得被研究


good_to_research('卧龙牧原1场3区生长场', '2020-10-31')

True

In [452]:
# 从starttime之后以每周为单位调取每周记录的频率，输出该场区的连续记录时间。比如11-01到11-08，中间可能有03号或者04号缺失数据但是不要紧。
#输出最后一次连续记录的时间点
#例子：卧龙牧原1场3区生长场， 爆发了一次，起始时间是2020-10-31, 结束时间是2020-11-08,并且结束时间后30天内不再有损失，那么清零时间为2020-11-08
def boom_starttime(fieldname, time_edge):
    '''输出这一轮的开始爆发时间
    time_edge: 应该是上一轮的截止时间。如果是第一轮，那么等于2020-10-31'''
    part_data = benifit_list_year.loc[(benifit_list_year.分场 == fieldname)&(benifit_list_year.业务日期 >= time_edge)]
    
    if not part_data.empty:
        return part_data.iloc[0,-1]

boom_starttime('卧龙牧原1场3区生长场', '2020-10-31')

'2020-10-31'

In [479]:
#输出这一轮的爆发结束时间点和清零时间点
def get_round_timelist(fieldname, round_starttime): 
    '''输出该轮的爆发结束时间， 和清零时间点
    round_starttime:该轮的开始时间'''
    part_data = benifit_list_year.loc[(benifit_list_year.分场 == fieldname)&(benifit_list_year.业务日期 >= round_starttime)]
    
    in_round = True #是否还在爆发时间段内
    round_starttime, round_endtime = round_starttime, (datetime.strptime(round_starttime, "%Y-%m-%d") + timedelta(days = 7)).strftime("%Y-%m-%d") 
    #每隔7天循环，直到找到爆发时间点
    while in_round:
        round_data = part_data.loc[(part_data.业务日期 > round_starttime)&(part_data.业务日期 <= round_endtime)] 
        if round_data.shape[0] == 0 : #如果七天内没有数据，说明已经停止爆发了
            round_finishtime = round_starttime #爆发结束点
            in_round = False 
        else: #爆发结束时间点未找到，继续循环
            round_starttime = round_data.iloc[-1,-1] #这七天记录的最后一个记录时间更新为下一个七天的开始时间
            round_endtime = (datetime.strptime(round_starttime, "%Y-%m-%d") + timedelta(days = 7)).strftime("%Y-%m-%d") #结束时间依旧是7天后
    
    #找到清零时间
    is_empty = True
    empty_start, empty_end = round_finishtime, (datetime.strptime(round_finishtime, "%Y-%m-%d") + timedelta(days = 30)).strftime("%Y-%m-%d") #30天内是否有记录
    while is_empty:
        quarantine_data = part_data.loc[(part_data.业务日期 <= empty_end)&(part_data.业务日期 > empty_start)]
        if quarantine_data.shape[0] == 0:
            empty_time = empty_end
            is_empty = False 
        else: #如果有记录，更新隔离时间
            empty_start = quarantine_data.iloc[-1,-1]
            empty_end = (datetime.strptime(empty_start, "%Y-%m-%d") + timedelta(days = 30)).strftime("%Y-%m-%d")
            #特殊情况
            if empty_end > '2021-12-30': #如果最后一天已经超出了利润表的最新时间，则输出empty_end且终止循环
                empty_time = empty_end
                is_empty = False 
            
    return [round_finishtime, empty_time]



get_round_timelist('卧龙牧原1场3区生长场', '2020-10-31')

['2020-11-08', '2020-12-08']

汇总函数

In [21]:
def get_label(fieldname,starttime, endtime):
    '''汇总函数，输出一个dataframe记录每轮的爆发时间和清零时间'''
    
    #制作dataframe 
    output_df = pd.DataFrame(columns = ['场区名', '爆发时间', '清零时间'])
    
    def good_to_research(fieldname, starttime, endtime):
        '''判断该场区是否值得被研究。10月31号之前没有记录才能被研究，并且之后有数据'''
        part_data = benifit_list_year.loc[(benifit_list_year.业务日期 < starttime) &(benifit_list_year.业务日期 >= endtime )& (benifit_list_year.分场  == fieldname)] #31号前没数据
        part2_data = benifit_list_year.loc[(benifit_list_year.业务日期 >= starttime) & (benifit_list_year.分场  == fieldname)] #31号后有数据
        return (part_data.empty & (not part2_data.empty) )#输出是否值得被研究
    
    #每轮爆发时间
    def boom_starttime(fieldname, time_edge):
        '''输出这一轮的开始爆发时间
        time_edge: 应该是上一轮的截止时间。如果是第一轮，那么等于2020-10-31'''
        part_data = benifit_list_year.loc[(benifit_list_year.分场 == fieldname)&(benifit_list_year.业务日期 >= time_edge)]

        if not part_data.empty:
            return part_data.iloc[0,-1]
    
    #每轮结束时间点 & 清零时间点
    def get_round_timelist(fieldname, round_starttime): 
        '''输出该轮的爆发结束时间， 和清零时间点
        round_starttime:该轮的开始时间'''
        part_data = benifit_list_year.loc[(benifit_list_year.分场 == fieldname)&(benifit_list_year.业务日期 >= round_starttime)]

        in_round = True #是否还在爆发时间段内
        round_starttime, round_endtime = round_starttime, (datetime.strptime(round_starttime, "%Y-%m-%d") + timedelta(days = 7)).strftime("%Y-%m-%d") 
        #每隔7天循环，直到找到爆发时间点
        while in_round:
            round_data = part_data.loc[(part_data.业务日期 > round_starttime)&(part_data.业务日期 <= round_endtime)] 
            if round_data.shape[0] == 0 : #如果七天内没有数据，说明已经停止爆发了
                round_finishtime = round_starttime #爆发结束点
                in_round = False 
            else: #爆发结束时间点未找到，继续循环
                round_starttime = round_data.iloc[-1,-1] #这七天记录的最后一个记录时间更新为下一个七天的开始时间
                round_endtime = (datetime.strptime(round_starttime, "%Y-%m-%d") + timedelta(days = 7)).strftime("%Y-%m-%d") #结束时间依旧是7天后
        #找到清零时间
        is_empty = True
        empty_start, empty_end = round_finishtime, (datetime.strptime(round_finishtime, "%Y-%m-%d") + timedelta(days = 30)).strftime("%Y-%m-%d") #30天内是否有记录
        while is_empty:
            quarantine_data = part_data.loc[(part_data.业务日期 <= empty_end)&(part_data.业务日期 > empty_start)]
            if quarantine_data.shape[0] == 0:
                empty_time = empty_end
                is_empty = False 
            else: #如果有记录，更新隔离时间
                empty_start = quarantine_data.iloc[-1,-1]
                empty_end = (datetime.strptime(empty_start, "%Y-%m-%d") + timedelta(days = 30)).strftime("%Y-%m-%d")
                #特殊情况
                if empty_end > '2021-12-30': #如果最后一天已经超出了利润表的最新时间，则输出empty_end且终止循环
                    empty_time = empty_end
                    is_empty = False 
        return [round_finishtime, empty_time]
    
    
    
    round_starttime_list = [] #每轮爆发时间列表
    round_emptytime_list = [] #每轮清零时间列表  
    
    if good_to_research(fieldname, starttime, endtime):
        #第一轮
        time_edge = starttime
        is_end = True 
        while is_end:
            if not benifit_list_year.loc[(benifit_list_year.业务日期 > time_edge) & (benifit_list_year.分场  == fieldname)].empty:
                thisround_starttime = boom_starttime(fieldname, time_edge) #第一轮爆发时间
                round_starttime_list.append(thisround_starttime)
                thisround_endtime, thisround_emptytime = get_round_timelist(fieldname, thisround_starttime)[0], get_round_timelist(fieldname, thisround_starttime)[1]
                round_emptytime_list.append(thisround_emptytime)
                time_edge = thisround_emptytime #更新时间节点，来预测下一轮的爆发时间
            else: #若不再有数据则停止
                is_end = False   
          
        #更新表格
        output_df['爆发时间'] = round_starttime_list
        output_df['清零时间'] = round_emptytime_list
        output_df['场区名'] = fieldname #放在最后面才可以自动填充所有行
            
    return output_df

    

            
        
    
    
    

In [22]:
boom_df = pd.DataFrame(columns = ['场区名', '爆发时间', '清零时间'])

for field in list(benifit_list_year.分场.unique()):
    part_df = get_label(field, '2020-11-31', '2020-11-01')
    if not part_df.empty:
        boom_df = pd.concat([boom_df,part_df],  axis=0, ignore_index = True)
    
boom_df  

,场区名,爆发时间,清零时间
0,大安牧原1场1区生长场,2020-12-13,2021-02-20
1,宁陵牧原8场繁殖场,2021-02-01,2021-04-01
2,内乡牧原20场2区生长场,2021-02-22,2021-03-31
3,代县牧原2场生长场,2020-12-23,2021-01-22
4,石首牧原6场繁殖场,2020-12-02,2021-02-04
...,...,...,...
813,汝州牧原综合体2场,2021-12-20,2022-01-28
814,横县牧原4场繁殖场,2021-12-21,2022-01-27
815,内乡牧原综合体2场,2021-12-21,2022-01-28
816,内乡牧原综合体13场,2021-12-27,2022-01-28


# 发病模型
- 输入正样本：该场区哺乳仔猪检出PED的前15天 （包括第15天）
- 输出负样本：该场区哺乳仔猪无检出PED（不包括正样本的其他样本）
- 适用场景：预警该场区当前时间是否为发病场区(场区有PED病毒但不一定造成损失，也不一定呈现PED阳性)

抗原筛选条件：
1、送检目的为病原流调，出生监控，断奶监控，疾病诊断，死猪监控，转群监控
2、以采样日期为准
3、工段限制：泌乳母猪13910、哺乳仔猪13905


## 贴标签，生成训练集

In [20]:
# 新宏信数据源（不准确）
# sql = '''SELECT * FROM 
# (SELECT DISTINCT c.batch_no, left(b.sampling_time,10) AS sampling_time, 
# a.project_code, a.project_name, a.result_determine, 
# a.case_no, a.sample_no, a.inspection_purpose_code, a.inspection_purpose_name
# FROM mydw.source_my_hx_result_pcr a
# LEFT JOIN mydw.source_my_hx_sampling_sample c ON a.sample_no = c.sample_no 
# LEFT JOIN mydw.source_my_hx_veterinary_sampling b ON b.id = a.sampling_id ) pcr 
# WHERE pcr.inspection_purpose_code IN (261332, 261314, 261301,261306,261302,261321,261331,261313,261304)
# AND pcr.result_determine = '阳性' AND pcr.project_code = 26120250 and (pcr.sampling_time between '2021-01-01' and '2021-12-31')'''

# hiveConn = hive.Connection(host='10.106.20.15', port=10000, username='szchenye', password='szcy230#',
#                                        database='mydw', auth='CUSTOM')
# ped_df = pd.read_sql(sql, hiveConn)

# 新宏信定量pcr取数
# sql = '''SELECT * FROM (
# SELECT DISTINCT c.batch_no, left(b.sampling_time, 10) AS sampling_time, 
# a.project_code, a.project_name, a.result_determine, a.sample_no, 
# a.inspection_purpose_code, a.inspection_purpose_name
# FROM mydw.source_my_hx_result_quantitative_pcr a 
# LEFT JOIN mydw.source_my_hx_sampling_sample c ON a.sample_no = c.sample_no 
# LEFT JOIN mydw.source_my_hx_veterinary_sampling b ON b.id = a.sampling_id 
# ) pcrdl 
# WHERE pcrdl.inspection_purpose_code IN (261332, 261314, 261301,261306,261302,261321,261331,261313,261304)
# AND pcrdl.result_determine = '阳性' AND pcrdl.project_code = 26120250 and (pcrdl.sampling_time between '2021-01-01' and '2021-12-31')
# '''

# hiveConn = hive.Connection(host='10.106.20.15', port=10000, username='szchenye', password='szcy230#',
#                                        database='mydw', auth='CUSTOM')
# peddl_df = pd.read_sql(sql, hiveConn)

In [4]:
#BI系统数据源
ped_data = pd.read_excel('PCR2021.xlsx')

In [5]:
# 选取场区名和采样时间（删除重复）
ped_field = ped_data.loc[:,['场次', '采样时间']].drop_duplicates(keep = 'first').sort_values(by = '采样时间').reset_index(drop = True) 

In [6]:
ped_field

,场次,采样时间
0,海兴牧原2场繁殖场,2021-01-01
1,唐河牧原5场繁殖场,2021-01-01
2,濉溪牧原11场繁殖场,2021-01-01
3,颍上牧原2场1区繁殖场,2021-01-01
4,濉溪牧原2场2区繁殖场,2021-01-01
...,...,...
2074,内乡牧原综合体10场,2022-01-02
2075,平原牧原1场繁殖场,2022-01-03
2076,汝州牧原综合体2场,2022-01-03
2077,林甸牧原1场繁殖场,2022-01-03


In [7]:
def sickmodel_label(fieldname):
    '''给发病模型贴标签。正样本为该场区检出PED前15天（包括检出当天）， 负样本为其他时间
    输入：场区名
    输出：dataframe, 该场区2021-01-01 到2022-01-03每天的标签'''
    
    #首先确定输出dataframe 
    output_df = pd.DataFrame(columns = ['场区名', '当前时间', '样本标签'])
    #首先填充当前时间，2021-01-01到2022-01-03一年时间
    time_list_1y = [(datetime.strptime('2021-01-01', "%Y-%m-%d") + timedelta(days = i)).strftime("%Y-%m-%d")  for i in range(0,368)]
    output_df['当前时间'] = time_list_1y
    output_df['场区名'] = fieldname
    
    def fill_positive_sample():
        '''填充正样本'''
        field_data = ped_field.loc[ped_field.场次 == fieldname]
        #采样时间列表
        positive_timelist = list(field_data.采样时间)
        
        #找到每个采样时间前15天
        for time in positive_timelist:
            starttime, endtime  = (datetime.strptime(time, "%Y-%m-%d") - timedelta(days = 14)).strftime("%Y-%m-%d") , time 
            #填充output_df,相应时间标签为1
            output_df.loc[(output_df.当前时间 <= endtime) &(output_df.当前时间 >= starttime), '样本标签'] = 1
        return output_df 
    
    #填充完正样本为1后，其他时间为负样本，标为0
    output_df = fill_positive_sample()
    return output_df.fillna(0)


sickmodel_label('海兴牧原2场繁殖场')

,场区名,当前时间,样本标签
0,海兴牧原2场繁殖场,2021-01-01,1
1,海兴牧原2场繁殖场,2021-01-02,0
2,海兴牧原2场繁殖场,2021-01-03,0
3,海兴牧原2场繁殖场,2021-01-04,0
4,海兴牧原2场繁殖场,2021-01-05,0
...,...,...,...
363,海兴牧原2场繁殖场,2021-12-30,0
364,海兴牧原2场繁殖场,2021-12-31,0
365,海兴牧原2场繁殖场,2022-01-01,0
366,海兴牧原2场繁殖场,2022-01-02,0


In [8]:
# 给ped_field内所有场区贴标签，得到训练集
field_list = list(ped_field.场次.unique())
labelled_data = sickmodel_label(field_list[0]) #开头不能为空

for i in range(1,len(field_list)):
    field = field_list[i]
    labelled_data = labelled_data.append(sickmodel_label(field), ignore_index = True)


In [9]:
labelled_data.样本标签.value_counts()

0    118975
1     13505
Name: 样本标签, dtype: int64

In [10]:
labelled_data.shape

(132480, 3)

## 训练模型
https://zhuanlan.zhihu.com/p/70602209

In [11]:
dataset = pd.read_csv('21_01m_22_01m_data.csv')
dataset = dataset.iloc[:, 1:] #去掉 unname 那一列

In [12]:
#训练数据
train_set = dataset.merge(labelled_data, how = 'left', on = ['场区名', '当前时间']).dropna(how = 'any').reset_index(drop = True)
#训练集和验证集
train_xin = train_set.loc[train_set.当前时间 <= '2021-11-01']
valid_xin = train_set.loc[train_set.当前时间 > '2021-12-01']

In [19]:
#train_set.to_csv('train_set.csv', index = False)

In [13]:
train_xin.样本标签.value_counts()

0.0    91059
1.0    11915
Name: 样本标签, dtype: int64

In [14]:
train_xin.columns

Index(['区域名', '子公司名', '场区名', '场区类型', '当前时间', '风险等级', '原因', '该场区哺乳猪群PED阳性单元数',
       '是否为PED阳性场区', '产房检出频数(同一天检出的归为一次)', '上游母猪是否为阳性单元/场区', '当前时间子公司内阳性场区占比',
       '该场区中和抗体合格率', '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)',
       '在群产房批次的上游怀孕批次是否返饲(返饲比例)', '在群产房批次的上游后备批次是否返饲(返饲比例)', '时间维度', '样本标签'],
      dtype='object')

In [15]:
#由于标签是高度不平衡的，所以我们要使得标签是平衡的以便获得标签的正态分布

#在创建子数据集之前打乱数据
train_xin = train_xin.sample(frac=1)

#正样本是11915个
positive_sample = train_xin.loc[train_xin.样本标签 == 1]
nagetive_sample = train_xin.loc[train_xin.样本标签 == 0][:11915]

normal_distributer_df = pd.concat([positive_sample, nagetive_sample])


#Shuffle dafaframe rows 打乱顺序
new_df = normal_distributer_df.sample(frac=1, random_state=42)

new_df.head()

,区域名,子公司名,场区名,场区类型,当前时间,风险等级,原因,该场区哺乳猪群PED阳性单元数,是否为PED阳性场区,产房检出频数(同一天检出的归为一次),上游母猪是否为阳性单元/场区,当前时间子公司内阳性场区占比,该场区中和抗体合格率,本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场),在群产房批次的上游怀孕批次是否返饲(返饲比例),在群产房批次的上游后备批次是否返饲(返饲比例),时间维度,样本标签
43666,安徽省,蒙城县,蒙城牧原6场繁殖场,繁殖场,2021-05-14,中高风险,场区怀孕及后备批次返饲比例<50%,未送检,否,未送检,未送检,26.35%,50.0%,15.0,100.0%,25.0%,30,0.0
57438,河南省,上蔡县,上蔡牧原7场繁殖场,繁殖场,2021-06-24,中低风险,当前时间子公司内20%以下为发病场区,0.0,否,0.0,否,6.06%,84.0%,无数据,66.67%,77.78%,30,0.0
71082,山东省,牡丹区,牡丹牧原7场繁殖场,繁殖场,2021-08-02,中高风险,本场区为阳性场区,0.0,是,0.0,未送检,7.46%,17.78%,9.0,0.0%,0.0%,30,0.0
91329,山东省,牡丹区,牡丹牧原7场繁殖场,繁殖场,2021-09-29,中高风险,本场区为阳性场区,0.0,是,0.0,否,11.11%,54.55%,9.0,0.0%,0.0%,30,0.0
3830,内蒙古自治区,开鲁,开鲁牧原5场繁殖场,繁殖场,2021-01-13,中高风险,场区怀孕及后备批次返饲比例<50%,0.0,否,0.0,未送检,5.88%,未送检,无数据,0.0%,0.0%,30,1.0


In [16]:
#new_df.to_csv('new_df.csv', index = False)

In [252]:
X_train_data = data_preprocess(new_df)
y_train_data = list(new_df.样本标签) + [0] #增加了一行，所以要增加一个标签y

#拟合模型
from sklearn.linear_model import LogisticRegression
lr_l1 = LogisticRegression()
lr_l1.fit(X_train_data, y_train_data)

LogisticRegression()

In [269]:
X_valid = data_preprocess(valid_xin)
y_valid = list(valid_xin.样本标签) + [0] 
y_pred_valid = lr_l1.predict(X_valid)
y_valid_prob = lr_l1.predict_proba(X_valid)[:,1]

from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

print('正确率',accuracy_score(y_valid,y_pred_valid))
print('精准率',precision_score(y_valid,y_pred_valid))
print('召回率',recall_score(y_valid,y_pred_valid))
print('调和平均值F1',f1_score(y_valid,y_pred_valid))
[fpr, tpr, thr] = roc_curve(y_valid, y_valid_prob)
print('AUC', auc(fpr, tpr))
print('对数损失', log_loss(y_valid, y_valid_prob))

正确率 0.741038961038961
精准率 0.13925645872715817
召回率 0.630527817403709
调和平均值F1 0.22812903225806452
AUC 0.7739643233814355
对数损失 0.5174074658050953


In [263]:
# 输出预测结果
def model2_0(test_data):
    '''模型2.0预测结果'''
    test_data_copy  = test_data.copy()
    test_data_copy = test_data_copy.loc[test_data_copy.是否为PED阳性场区 != '未送检']

    X_test = data_preprocess(test_data_copy)
    #预测结果
    y_test_prob = lr_l1.predict_proba(X_test)
    
    #添加字段 PED预测概率
    test_data_copy.loc[test_data_copy.是否为PED阳性场区 != '未送检', '预测PED概率'] = y_test_prob[:-1, 1] #去除最后一行附加场区的输出， 保持格式一致
    test_data_copy =  test_data_copy.sort_values(by = ['预测PED概率'],ascending= False )
    
    # 0.76以上为高风险
    test_data_copy.loc[test_data_copy.预测PED概率 > 0.76, '预测风险'] = '有风险'
    test_data_copy.预测风险 = test_data_copy.预测风险.fillna('无风险')
    
    return test_data_copy

aa = model2_0(valid_xin)
aa

,区域名,子公司名,场区名,场区类型,当前时间,风险等级,原因,该场区哺乳猪群PED阳性单元数,是否为PED阳性场区,产房检出频数(同一天检出的归为一次),上游母猪是否为阳性单元/场区,当前时间子公司内阳性场区占比,该场区中和抗体合格率,本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场),在群产房批次的上游怀孕批次是否返饲(返饲比例),在群产房批次的上游后备批次是否返饲(返饲比例),时间维度,样本标签,预测PED概率,预测风险
124631,湖北省,黄梅县,黄梅牧原2场繁殖场,繁殖场,2022-01-02,高风险,在群哺乳仔猪累计阳性单元大于1,5.0,是,5.0,是,72.58%,10.0%,6.0,77.78%,0.0%,30,0.0,0.930379,有风险
124981,湖北省,黄梅县,黄梅牧原2场繁殖场,繁殖场,2022-01-03,高风险,在群哺乳仔猪累计阳性单元大于1,5.0,是,5.0,是,74.58%,10.0%,6.0,63.64%,0.0%,30,0.0,0.930379,有风险
124754,内蒙古自治区,奈曼,奈曼牧原3场2区繁殖场,繁殖场,2022-01-03,高风险,在群哺乳仔猪累计阳性单元大于1,5.0,是,5.0,是,55.67%,83.33%,76.0,75.0%,0.0%,30,0.0,0.924814,有风险
121884,河南省,育种内乡,内乡牧原种猪育种26场繁殖场,繁殖场,2021-12-26,高风险,在群哺乳仔猪累计阳性单元大于1,5.0,是,11.0,是,40.0%,33.33%,88.0,66.67%,0.0%,30,0.0,0.924814,有风险
123021,广西,宾阳县,宾阳牧原6场繁殖场,繁殖场,2021-12-29,高风险,当前时间子公司内20%以上场区为发病场区，且场区未返饲,1.0,是,1.0,是,41.67%,未送检,无数据,0.0%,无数据,30,1.0,0.922951,有风险
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115888,山西省,万荣,万荣牧原7场综合场,综合场,2021-12-09,中低风险,当前时间子公司内20%以下为发病场区,未送检,否,未送检,未送检,13.48%,未送检,无数据,无数据,无数据,30,0.0,0.048368,无风险
120792,山西省,万荣,万荣牧原7场综合场,综合场,2021-12-23,中低风险,当前时间子公司内20%以下为发病场区,未送检,否,未送检,未送检,11.68%,未送检,无数据,无数据,无数据,30,0.0,0.048368,无风险
120441,山西省,万荣,万荣牧原7场综合场,综合场,2021-12-22,中低风险,当前时间子公司内20%以下为发病场区,未送检,否,未送检,未送检,11.63%,未送检,无数据,无数据,无数据,30,0.0,0.048368,无风险
113758,辽宁省,阜新,阜新牧原3场1区繁殖场,繁殖场,2021-12-02,中高风险,场区怀孕及后备批次返饲比例<50%,未送检,否,未送检,未送检,15.44%,未送检,无数据,无数据,0.0%,30,0.0,0.037763,无风险


In [264]:
aa.loc[(aa.样本标签 == 1)&(aa.预测风险 == '有风险')].shape #Tp

(222, 20)

In [257]:
aa.loc[(aa.样本标签 == 1)].shape 

(627, 20)

In [265]:
aa.loc[aa.预测风险 == '有风险'].shape

(634, 20)

In [224]:
valid_xin.shape

(11549, 18)

In [228]:
valid_xin.场区名.unique().shape

(351,)

In [266]:
222/634

0.3501577287066246

In [267]:
222/627

0.35406698564593303

In [180]:
train_set.shape

(124991, 18)

In [181]:
dataset.shape

(483129, 17)

In [182]:
valid_xin.shape

(11549, 18)

In [251]:
def data_preprocess(train_data):
    '''离散化数据，为准备拟合模型和准备预测'''
    
    from sklearn.preprocessing import KBinsDiscretizer
    data_process = train_data.copy()
    #data_process = data_process.loc[data_process.is_ped.notnull()]
    #附加一行，为了避免离散化后的特征不一致
    addtion_row = {'场区名': '附加', '该场区哺乳猪群PED阳性单元数':'未送检','当前时间子公司内阳性场区占比':'未送检', '该场区中和抗体合格率':'未送检', 
           '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)':'无数据','在群产房批次的上游怀孕批次是否返饲(返饲比例)':'无数据', 
           '在群产房批次的上游后备批次是否返饲(返饲比例)':'无数据','是否为PED阳性场区':'未送检','上游母猪是否为阳性单元/场区':'否',
           '产房检出频数(同一天检出的归为一次)':'0.0', '该场区哺乳猪群PED阳性单元数': '0.0'}
    data_process = data_process.append(addtion_row, ignore_index = True)

    #当前时间子公司内阳性场区占比
    aa = data_process.loc[data_process.当前时间子公司内阳性场区占比 != '未送检'].当前时间子公司内阳性场区占比.str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est = KBinsDiscretizer(n_bins = 5, encode='ordinal', strategy='uniform')
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process.当前时间子公司内阳性场区占比 != '未送检', '当前时间子公司内阳性场区占比'] = Xt.astype(str)

    #产房检出频数(同一天检出的归为一次)
    data_process.loc[~data_process['该场区哺乳猪群PED阳性单元数'].isin(['未送检', '0.0']), '该场区哺乳猪群PED阳性单元数'] = '>=0.0' #~表示反向函数，即提取不在这个列表的行
    data_process.loc[~data_process['产房检出频数(同一天检出的归为一次)'].isin(['未送检', '0.0']), '产房检出频数(同一天检出的归为一次)'] = '>=0.0'

    #该场区中和抗体合格率
    aa = data_process.loc[data_process['该场区中和抗体合格率'] != '未送检']['该场区中和抗体合格率'].str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['该场区中和抗体合格率'] != '未送检', '该场区中和抗体合格率'] = Xt.astype(str)

    #本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)
    aa = data_process.loc[data_process['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] != '无数据']['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)']
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] != '无数据', '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] = Xt.astype(str)

    #在群产房批次的上游怀孕批次是否返饲(返饲比例)
    aa = data_process.loc[data_process['在群产房批次的上游怀孕批次是否返饲(返饲比例)'] != '无数据']['在群产房批次的上游怀孕批次是否返饲(返饲比例)'].str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['在群产房批次的上游怀孕批次是否返饲(返饲比例)'] != '无数据','在群产房批次的上游怀孕批次是否返饲(返饲比例)'] = Xt.astype(str)

    #在群产房批次的上游后备批次是否返饲(返饲比例)
    aa = data_process.loc[data_process['在群产房批次的上游后备批次是否返饲(返饲比例)'] != '无数据']['在群产房批次的上游后备批次是否返饲(返饲比例)'].str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['在群产房批次的上游后备批次是否返饲(返饲比例)'] != '无数据','在群产房批次的上游后备批次是否返饲(返饲比例)'] = Xt.astype(str)


    X_data = data_process.loc[:, ['该场区哺乳猪群PED阳性单元数', '是否为PED阳性场区', '产房检出频数(同一天检出的归为一次)',
                                '上游母猪是否为阳性单元/场区', '当前时间子公司内阳性场区占比', '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)', 
                               '在群产房批次的上游怀孕批次是否返饲(返饲比例)', '在群产房批次的上游后备批次是否返饲(返饲比例)']]

    #独热编码
    from sklearn.feature_extraction import DictVectorizer
    vect = DictVectorizer()
    X_data = vect.fit_transform(X_data.to_dict(orient="records"))
    
    return X_data

X_train_data = data_preprocess(train_xin)
y_train_data = list(train_xin.样本标签) + [0] #增加了一行，所以要增加一个标签y

#拟合模型
from sklearn.linear_model import LogisticRegression
lr_l1 = LogisticRegression()
lr_l1.fit(X_train_data, y_train_data)

# 输出预测结果
def model2_0(test_data):
    '''模型2.0预测结果'''
    test_data_copy  = test_data.copy()
    test_data_copy = test_data_copy.loc[test_data_copy.是否为PED阳性场区 != '未送检']

    X_test = data_preprocess(test_data_copy)
    #预测结果
    y_test_prob = lr_l1.predict_proba(X_test)
    
    #添加字段 PED预测概率
    test_data_copy.loc[test_data_copy.是否为PED阳性场区 != '未送检', '预测PED概率'] = y_test_prob[:-1, 1] #去除最后一行附加场区的输出， 保持格式一致
    test_data_copy =  test_data_copy.sort_values(by = ['预测PED概率'],ascending= False )
    
    # 0.76以上为高风险
    test_data_copy.loc[test_data_copy.预测PED概率 > 0.7, '预测风险'] = '有风险'
    test_data_copy.预测风险 = test_data_copy.预测风险.fillna('无风险')
    
    return test_data_copy

/data/envir/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
X_valid = data_preprocess(valid_xin)
y_valid = list(valid_xin.样本标签) + [0] 
y_pred_valid = lr_l1.predict(X_valid)
y_valid_prob = lr_l1.predict_proba(X_valid)[:,1]

from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

print('正确率',accuracy_score(y_valid,y_pred_valid))
print('精准率',precision_score(y_valid,y_pred_valid))
print('召回率',recall_score(y_valid,y_pred_valid))
print('调和平均值F1',f1_score(y_valid,y_pred_valid))
[fpr, tpr, thr] = roc_curve(y_valid, y_valid_prob)
print('AUC', auc(fpr, tpr))
print('对数损失', log_loss(y_valid, y_valid_prob))

## 特征工程
由于上模型欠拟合。需要找到新的特征，检查其与标签的相似程度

In [34]:
train_set

,区域名,子公司名,场区名,场区类型,当前时间,风险等级,原因,该场区哺乳猪群PED阳性单元数,是否为PED阳性场区,产房检出频数(同一天检出的归为一次),上游母猪是否为阳性单元/场区,当前时间子公司内阳性场区占比,该场区中和抗体合格率,本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场),在群产房批次的上游怀孕批次是否返饲(返饲比例),在群产房批次的上游后备批次是否返饲(返饲比例),时间维度,样本标签
0,山西省,万荣,万荣牧原2场繁殖场,繁殖场,2021-01-01,高风险,当前时间子公司内20%以上场区为发病场区，且场区未返饲,0.0,是,0.0,否,47.22%,42.42%,无数据,21.05%,0.0%,30,1.0
1,山西省,万荣,万荣牧原3场繁殖场,繁殖场,2021-01-01,中高风险,本场区为阳性场区,未送检,是,未送检,未送检,47.22%,未送检,71.0,73.33%,80.0%,30,0.0
2,山西省,万荣,万荣牧原5场繁殖场,繁殖场,2021-01-01,高风险,在群哺乳仔猪累计阳性单元大于1,7.0,是,10.0,未送检,47.22%,未送检,无数据,5.0%,16.67%,30,1.0
3,山西省,万荣,万荣牧原7场综合场,综合场,2021-01-01,高风险,当前时间子公司内20%以上场区为发病场区，且场区未返饲,未送检,未送检,未送检,未送检,47.22%,未送检,无数据,0.0%,0.0%,30,0.0
4,河南省,上蔡县,上蔡牧原12场1区繁殖场,繁殖场,2021-01-01,中高风险,场区怀孕及后备批次返饲比例<50%,0.0,否,0.0,未送检,0.0%,39.39%,无数据,7.14%,50.0%,30,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124986,黑龙江省,龙江,龙江牧原1场1区综合场,综合场,2022-01-03,中高风险,本场区为阳性场区,0.0,是,0.0,未送检,32.14%,未送检,54.0,0.0%,28.57%,30,0.0
124987,黑龙江省,龙江,龙江牧原1场2区综合场,综合场,2022-01-03,高风险,当前时间子公司内20%以上场区为发病场区，且场区未返饲,0.0,是,0.0,未送检,32.14%,未送检,无数据,0.0%,0.0%,30,0.0
124988,黑龙江省,龙江,龙江牧原1场3区综合场,综合场,2022-01-03,高风险,当前时间子公司内20%以上场区为发病场区，且场区未返饲,未送检,否,未送检,未送检,32.14%,未送检,无数据,40.0%,0.0%,30,0.0
124989,黑龙江省,龙江,龙江牧原1场4区综合场,综合场,2022-01-03,高风险,当前时间子公司内20%以上场区为发病场区，且场区未返饲,0.0,是,0.0,未送检,32.14%,未送检,无数据,0.0%,0.0%,30,0.0


上轮检出阳性时间间隔。场区上一次检出阳性单元的采样时间距离当前时间的间隔（采样时间在当前时间前）

In [53]:
sql = '''select * from myana.datamining_DWS_DS_antigen_UD where sampling_time between '2021-01-01' and '2022-01-03' '''
hiveConn = hive.Connection(host='10.106.20.15', port=10000, username='szchenye', password='szcy230#',
                                       database='mydw', auth='CUSTOM')
df = pd.read_sql(sql, hiveConn)
df.sort_values(by = 'sampling_time')

,batch_no,unit,pregnancy_age,sampling_time,pcr_result,pcrdl_result
165591,BS0000136236032201225,3,3.0,2021-01-01 00:08:21.0,"[""梭菌""]",None
72911,JN0000436248903201208,26,187.0,2021-01-01 00:31:14.467,None,"[""蓝耳""]"
111544,FG610536179899201123,35,0.0,2021-01-01 01:03:04.977,None,"[""A""]"
221011,FG610536179899201123,35,33.0,2021-01-01 01:09:25.93,None,"[""A""]"
63795,TS230236221893201124,16,40.0,2021-01-01 01:15:59.0,None,"[""A""]"
...,...,...,...,...,...,...
183388,JZ00002050222211123088,88,63.0,2022-01-03,None,"[""蓝耳""]"
115145,TK3205268334210919002,2,90.0,2022-01-03,None,"[""A""]"
153788,SC00003020559211109038,38,142.0,2022-01-03,"[""副猪嗜血杆菌""]","[""副猪嗜血杆菌""]"
30328,GN5703023012211206002,4,107.0,2022-01-03,None,"[""A""]"


# B
●  模型B（现模型２.０）：输入每天场区表现，输出每天代表该天之后７天内PED检出概率值。
训练集：取2020-11到2021-3月每天的场区数据
验证集：2021-08到2021-09抗原表检出阳性场区和检出时间
正样本：根据抗原表，在当前时间后７天内检出PED阳性则为正样本
负样本：不包括正样本的样本
模型准确率＝所有场区预判正确的正样本之和／所有阳性场区实际检出天数之和


In [47]:
def isped(data,currenttime):
    '''几天内是否阳性——0或1'''

    isped_sql = '''SELECT A.*, B.ffield FROM 
    (
    SELECT ffieldid,sum(disease) AS is_ped
        FROM 
        (
        SELECT DISTINCT a.batch_no as fbatchno,b.ffieldid,b.fareaid,CASE WHEN (concat_ws(',',a.pcr_result) LIKE '%流行性腹泻%' 
        OR concat_ws(',',a.pcrdl_result) LIKE '%流行性腹泻%')THEN 1 ELSE 0 END AS disease
        FROM myana.datamining_DWS_DS_antigen_UD a
        LEFT JOIN myana.datamining_DWD_BI b ON a.batch_no=b.fbatchno
        WHERE datediff('{time}', a.`sampling_time`) <= 7 AND datediff('{time}', a.sampling_time) >= 0
        ) 
        GROUP BY ffieldid) A LEFT JOIN myana.datamining_DIM_AF B ON A.ffieldid = B.ffieldid 
    '''

    hiveConn = hive.Connection(host='10.106.20.15', port=10000, username='szchenye', password='szcy230#',
                                       database='mydw', auth='CUSTOM')

    isped_sql = isped_sql.format(time = currenttime)
    isped_df = pd.read_sql(isped_sql, hiveConn)
    #isped_df['时间维度'] = daygap
    
    #添加新字段
    ffield_list = list(isped_df.dropna(how = 'any').ffield.unique())
    
    for field in ffield_list: 
        num = isped_df.loc[isped_df.ffield == field].is_ped.values[0]
        if num == 0: 
            data.loc[(data.场区名 == field) & (data.当前时间 == currenttime), 'is_ped'] = 0
        else: 
            data.loc[(data.场区名 == field) & (data.当前时间 == currenttime), 'is_ped'] = 1
        
    return data


In [19]:
train_data = data_11_3m.copy() #训练集
valid_data = data_7_12m.copy() #验证集

#两个集合天贴上标签


In [50]:
# 贴上7天内是否检出PED标签
timelist_11_3m = [(datetime.strptime('2020-11-01', "%Y-%m-%d") + timedelta(days = i)).strftime("%Y-%m-%d") for i in range(1,151, 1)]
for currenttime in timelist_11_3m:
    train_data = isped(train_data,currenttime)

In [72]:
timelist_7_12m = [(datetime.strptime('2021-07-01', "%Y-%m-%d") + timedelta(days = i)).strftime("%Y-%m-%d") for i in range(1,176, 1)]
for currenttime in timelist_7_12m:
    valid_data = isped(valid_data,currenttime)

In [73]:
valid_data

,Unnamed: 0,区域名,子公司名,场区名,场区类型,当前时间,风险等级,原因,该场区哺乳猪群PED阳性单元数,是否为PED阳性场区,产房检出频数(同一天检出的归为一次),上游母猪是否为阳性单元/场区,当前时间子公司内阳性场区占比,该场区中和抗体合格率,本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场),在群产房批次的上游怀孕批次是否返饲(返饲比例),在群产房批次的上游后备批次是否返饲(返饲比例),时间维度,is_ped
0,0,山西省,万荣,万荣牧原10场生长场,生长场,2021-07-01,中低风险,当前时间子公司内20%以下为发病场区,未送检,否,未送检,未送检,1.28%,未送检,无数据,无数据,无数据,30,NaN
1,1,山西省,万荣,万荣牧原19场生长场,生长场,2021-07-01,中高风险,场区怀孕及后备批次返饲比例<50%,未送检,否,未送检,未送检,1.28%,未送检,无数据,无数据,0.0%,30,NaN
2,2,山西省,万荣,万荣牧原1场繁殖场,繁殖场,2021-07-01,中高风险,本场区为阳性场区,0.0,是,0.0,否,1.28%,80.0%,135.0,0.0%,0.0%,30,NaN
3,3,山西省,万荣,万荣牧原2场繁殖场,繁殖场,2021-07-01,中高风险,场区怀孕及后备批次返饲比例<50%,0.0,否,0.0,否,1.28%,50.0%,无数据,2.86%,8.33%,30,NaN
4,4,山西省,万荣,万荣牧原4场生长场,生长场,2021-07-01,中低风险,"场区已返饲,最近一次中抗检测合格率小于80%",未送检,否,未送检,未送检,1.28%,未送检,1189.0,无数据,无数据,30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249143,249143,黑龙江省,龙江,龙江牧原1场4区综合场,综合场,2021-12-24,中高风险,本场区为阳性场区,0.0,是,0.0,未送检,18.18%,未送检,无数据,0.0%,0.0%,30,NaN
249144,249144,黑龙江省,龙江,龙江牧原3场繁殖场,繁殖场,2021-12-24,中高风险,场区怀孕及后备批次返饲比例<50%,0.0,否,0.0,否,18.18%,未送检,2.0,0.0%,0.0%,30,NaN
249145,249145,黑龙江省,龙江,龙江牧原4场生长场,生长场,2021-12-24,中高风险,场区怀孕及后备批次返饲比例<50%,未送检,否,未送检,未送检,18.18%,未送检,无数据,无数据,0.0%,30,NaN
249146,249146,黑龙江省,龙江,龙江牧原7场生长场,生长场,2021-12-24,中高风险,场区怀孕及后备批次返饲比例<50%,未送检,否,未送检,未送检,18.18%,未送检,无数据,无数据,0.0%,30,NaN


In [76]:
# train_data.to_csv('train_data_B.csv', index = False)
# valid_data.to_csv('valid_data_B.csv', index = False)

## 拟合模型

In [ ]:
# train_data = pd.read_csv('train_data_B.csv')
# valid_data = pd.read_csv('valid_data_B.csv')

In [ ]:
test_data_B
train_data

In [384]:
data_process = train_data.copy()

In [368]:
#当前时间子公司内阳性场区占比
aa = data_process.loc[data_process.当前时间子公司内阳性场区占比 != '未送检'].当前时间子公司内阳性场区占比.str.split('%').apply(lambda x: float(x[0]))
aa = np.array(aa).reshape(-1,1)
est = KBinsDiscretizer(n_bins = 5, encode='ordinal', strategy='uniform')
est.fit(aa)
Xt = est.transform(aa)
data_process.loc[data_process.当前时间子公司内阳性场区占比 != '未送检', '当前时间子公司内阳性场区占比'] = Xt.astype(str)

In [327]:
#该场区中和抗体合格率
aa = data_process.loc[data_process['该场区中和抗体合格率'] != '未送检']['该场区中和抗体合格率'].str.split('%').apply(lambda x: float(x[0]))
aa = np.array(aa).reshape(-1,1)
est.fit(aa)
Xt = est.transform(aa)
data_process.loc[data_process['该场区中和抗体合格率'] != '未送检', '该场区中和抗体合格率'] = Xt.astype(str)

In [334]:
#本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)
aa = data_process.loc[data_process['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] != '无数据']['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)']
aa = np.array(aa).reshape(-1,1)
est.fit(aa)
Xt = est.transform(aa)
data_process.loc[data_process['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] != '无数据', '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] = Xt.astype(str)

In [341]:
#在群产房批次的上游怀孕批次是否返饲(返饲比例)
aa = data_process.loc[data_process['在群产房批次的上游怀孕批次是否返饲(返饲比例)'] != '无数据']['在群产房批次的上游怀孕批次是否返饲(返饲比例)'].str.split('%').apply(lambda x: float(x[0]))
aa = np.array(aa).reshape(-1,1)
est.fit(aa)
Xt = est.transform(aa)
data_process.loc[data_process['在群产房批次的上游怀孕批次是否返饲(返饲比例)'] != '无数据','在群产房批次的上游怀孕批次是否返饲(返饲比例)'] = Xt.astype(str)

In [347]:
#在群产房批次的上游后备批次是否返饲(返饲比例)
aa = data_process.loc[data_process['在群产房批次的上游后备批次是否返饲(返饲比例)'] != '无数据']['在群产房批次的上游后备批次是否返饲(返饲比例)'].str.split('%').apply(lambda x: float(x[0]))
aa = np.array(aa).reshape(-1,1)
est.fit(aa)
Xt = est.transform(aa)
data_process.loc[data_process['在群产房批次的上游后备批次是否返饲(返饲比例)'] != '无数据','在群产房批次的上游后备批次是否返饲(返饲比例)'] = Xt.astype(str)

In [377]:
est.bin_edges_

array([array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.])],
      dtype=object)

In [385]:
aa = data_process.loc[data_process.当前时间子公司内阳性场区占比 != '未送检'].当前时间子公司内阳性场区占比.str.split('%').apply(lambda x: float(x[0]))
aa = np.array(aa).reshape(-1,1)
est = KBinsDiscretizer(n_bins = 10, encode='ordinal', strategy='uniform')
est.fit(aa)
Xt = est.transform(aa)
Xt

array([[0.],
       [0.],
       [0.],
       ...,
       [4.],
       [4.],
       [4.]])

In [383]:
aa

array([[ 3.33],
       [ 3.33],
       [ 3.33],
       ...,
       [44.16],
       [44.16],
       [44.16]])

In [386]:
est.inverse_transform(Xt)

array([[ 5.],
       [ 5.],
       [ 5.],
       ...,
       [45.],
       [45.],
       [45.]])

In [59]:
def data_preprocess(train_data):
    '''离散化数据，为准备拟合模型和准备预测'''
    
    from sklearn.preprocessing import KBinsDiscretizer
    data_process = train_data.copy()
    #data_process = data_process.loc[data_process.is_ped.notnull()]
    #附加一行，为了避免离散化后的特征不一致
    addtion_row = {'场区名': '附加', '该场区哺乳猪群PED阳性单元数':'未送检','当前时间子公司内阳性场区占比':'未送检', '该场区中和抗体合格率':'未送检', 
           '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)':'无数据','在群产房批次的上游怀孕批次是否返饲(返饲比例)':'无数据', 
           '在群产房批次的上游后备批次是否返饲(返饲比例)':'无数据','是否为PED阳性场区':'未送检','上游母猪是否为阳性单元/场区':'否',
           '产房检出频数(同一天检出的归为一次)':'0.0', '该场区哺乳猪群PED阳性单元数': '0.0'}
    data_process = data_process.append(addtion_row, ignore_index = True)

    #当前时间子公司内阳性场区占比
    aa = data_process.loc[data_process.当前时间子公司内阳性场区占比 != '未送检'].当前时间子公司内阳性场区占比.str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est = KBinsDiscretizer(n_bins = 5, encode='ordinal', strategy='uniform')
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process.当前时间子公司内阳性场区占比 != '未送检', '当前时间子公司内阳性场区占比'] = Xt.astype(str)

    #产房检出频数(同一天检出的归为一次)
    data_process.loc[~data_process['该场区哺乳猪群PED阳性单元数'].isin(['未送检', '0.0']), '该场区哺乳猪群PED阳性单元数'] = '>=0.0' #~表示反向函数，即提取不在这个列表的行
    data_process.loc[~data_process['产房检出频数(同一天检出的归为一次)'].isin(['未送检', '0.0']), '产房检出频数(同一天检出的归为一次)'] = '>=0.0'

    #该场区中和抗体合格率
    aa = data_process.loc[data_process['该场区中和抗体合格率'] != '未送检']['该场区中和抗体合格率'].str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['该场区中和抗体合格率'] != '未送检', '该场区中和抗体合格率'] = Xt.astype(str)

    #本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)
    aa = data_process.loc[data_process['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] != '无数据']['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)']
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] != '无数据', '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)'] = Xt.astype(str)

    #在群产房批次的上游怀孕批次是否返饲(返饲比例)
    aa = data_process.loc[data_process['在群产房批次的上游怀孕批次是否返饲(返饲比例)'] != '无数据']['在群产房批次的上游怀孕批次是否返饲(返饲比例)'].str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['在群产房批次的上游怀孕批次是否返饲(返饲比例)'] != '无数据','在群产房批次的上游怀孕批次是否返饲(返饲比例)'] = Xt.astype(str)

    #在群产房批次的上游后备批次是否返饲(返饲比例)
    aa = data_process.loc[data_process['在群产房批次的上游后备批次是否返饲(返饲比例)'] != '无数据']['在群产房批次的上游后备批次是否返饲(返饲比例)'].str.split('%').apply(lambda x: float(x[0]))
    aa = np.array(aa).reshape(-1,1)
    est.fit(aa)
    Xt = est.transform(aa)
    data_process.loc[data_process['在群产房批次的上游后备批次是否返饲(返饲比例)'] != '无数据','在群产房批次的上游后备批次是否返饲(返饲比例)'] = Xt.astype(str)


    X_data = data_process.loc[:, ['该场区哺乳猪群PED阳性单元数', '是否为PED阳性场区', '产房检出频数(同一天检出的归为一次)',
                                '上游母猪是否为阳性单元/场区', '当前时间子公司内阳性场区占比', '本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场)', 
                               '在群产房批次的上游怀孕批次是否返饲(返饲比例)', '在群产房批次的上游后备批次是否返饲(返饲比例)']]

    #独热编码
    from sklearn.feature_extraction import DictVectorizer
    vect = DictVectorizer()
    X_data = vect.fit_transform(X_data.to_dict(orient="records"))
    
    return X_data

X_train_data = data_preprocess(train_data.loc[train_data.is_ped.notnull()])
y_train_data = list(train_data.loc[train_data.is_ped.notnull()].is_ped) + [0] #增加了一行，所以要增加一个标签y

#拟合模型
from sklearn.linear_model import LogisticRegression
lr_l1 = LogisticRegression()
lr_l1.fit(X_train_data, y_train_data)

# 输出预测结果
def model2_0(test_data):
    '''模型2.0预测结果'''
    test_data_copy  = test_data.copy()
    test_data_copy = test_data_copy.loc[test_data_copy.是否为PED阳性场区 != '未送检']

    X_test = data_preprocess(test_data_copy)
    #预测结果
    y_test_prob = lr_l1.predict_proba(X_test)
    
    #添加字段 PED预测概率
    test_data_copy.loc[test_data_copy.是否为PED阳性场区 != '未送检', '预测PED概率'] = y_test_prob[:-1, 1] #去除最后一行附加场区的输出， 保持格式一致
    test_data_copy =  test_data_copy.sort_values(by = ['预测PED概率'],ascending= False )
    
    # 0.76以上为高风险
    test_data_copy.loc[test_data_copy.预测PED概率 > 0.5, '预测风险'] = '有风险'
    test_data_copy.预测风险 = test_data_copy.预测风险.fillna('无风险')
    
    return test_data_copy

/data/envir/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [234]:
valid_data_B.is_ped.value_counts()

0.0    64284
1.0    14250
Name: is_ped, dtype: int64

In [387]:
valid_data_B = valid_data.loc[(valid_data.当前时间 < '2021-10-01')&(valid_data.is_ped.notnull())]
X_valid = data_preprocess(valid_data_B)
y_valid = list(valid_data_B.is_ped) + [0] 
y_pred_valid = lr_l1.predict(X_valid)
y_valid_prob = lr_l1.predict_proba(X_valid)[:,1]

In [232]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss

print('正确率',accuracy_score(y_valid,y_pred_valid))
print('精准率',precision_score(y_valid,y_pred_valid))
print('召回率',recall_score(y_valid,y_pred_valid))
print('调和平均值F1',f1_score(y_valid,y_pred_valid))
[fpr, tpr, thr] = roc_curve(y_valid, y_valid_prob)
print('AUC', auc(fpr, tpr))
print('对数损失', log_loss(y_valid, y_valid_prob))

正确率 0.835360030559623
精准率 0.529432750624331
召回率 0.8331228070175438
调和平均值F1 0.6474341495337296
AUC 0.9124170010466004
对数损失 0.24127376723507485


## 测评

In [390]:
test_data_B = valid_data.loc[valid_data.当前时间 > '2021-10-01'].fillna(0)
X_test = data_preprocess(test_data_B)
y_test_pred = lr_l1.predict_proba(X_test)

In [391]:
test_data_B['预测PED概率'] = y_test_pred[:-1,1]

In [392]:
#高风险场区
test_data_B.loc[test_data_B.场区名 == '前郭牧原4场1区繁殖场']

,Unnamed: 0,区域名,子公司名,场区名,场区类型,当前时间,风险等级,原因,该场区哺乳猪群PED阳性单元数,是否为PED阳性场区,产房检出频数(同一天检出的归为一次),上游母猪是否为阳性单元/场区,当前时间子公司内阳性场区占比,该场区中和抗体合格率,本场区返饲间隔(取该场区最近一次返饲时间距当前的时间间隔，只针对繁殖场),在群产房批次的上游怀孕批次是否返饲(返饲比例),在群产房批次的上游后备批次是否返饲(返饲比例),时间维度,is_ped,预测PED概率
129669,129669,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-10-02,高风险,在群哺乳仔猪累计阳性单元大于1,2.0,是,2.0,是,50.0%,未送检,81.0,0.0%,88.89%,30,1.0,0.858237
131088,131088,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-10-03,高风险,在群哺乳仔猪累计阳性单元大于1,3.0,是,3.0,是,48.21%,未送检,81.0,0.0%,88.89%,30,1.0,0.858237
132507,132507,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-10-04,高风险,在群哺乳仔猪累计阳性单元大于1,3.0,是,3.0,是,45.1%,未送检,81.0,0.0%,88.89%,30,1.0,0.858237
133926,133926,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-10-05,高风险,在群哺乳仔猪累计阳性单元大于1,3.0,是,3.0,是,45.1%,未送检,81.0,0.0%,87.5%,30,1.0,0.858237
135347,135347,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-10-06,高风险,在群哺乳仔猪累计阳性单元大于1,3.0,是,3.0,是,42.59%,未送检,81.0,0.0%,87.5%,30,1.0,0.858237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242205,242205,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-12-20,中高风险,本场区为阳性场区,0.0,是,0.0,否,3.41%,86.36%,81.0,100.0%,52.94%,30,0.0,0.390431
243639,243639,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-12-21,中高风险,本场区为阳性场区,0.0,是,0.0,否,2.97%,90.0%,81.0,100.0%,52.94%,30,0.0,0.390431
245073,245073,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-12-22,中高风险,本场区为阳性场区,0.0,是,0.0,否,2.88%,90.0%,81.0,100.0%,52.94%,30,0.0,0.390431
246506,246506,吉林省,前郭尔罗斯蒙古族自治县,前郭牧原4场1区繁殖场,繁殖场,2021-12-23,中高风险,本场区为阳性场区,0.0,是,0.0,否,3.03%,90.0%,81.0,50.0%,52.94%,30,0.0,0.484212


In [398]:
risk_field_raw = pd.read_excel('利润损失分析.xlsx')
risk_field = risk_field_raw.loc[risk_field_raw.业务日期 < '2021-12-01'] #只参考11月的爆发场区
TT = 0 
TF_TT = 0 
TT_FT = 0
for field in list(risk_field.分场.unique()):
    field_data = test_data_B.loc[test_data_B.场区名 == field]
    ped_time = risk_field.loc[risk_field.分场 == field].业务日期.values[0]
    
    starttime = (datetime.strptime(ped_time, "%Y-%m-%d") - timedelta(days = 7)).strftime("%Y-%m-%d")
    
    TT_list = list(field_data.loc[(field_data.当前时间 <= ped_time)&(field_data.当前时间 > starttime)& (field_data.预测PED概率 > 0.5)].is_ped) #此段时间是否检出PED 且预测成功 即TT
    is_ped_list = list(field_data.loc[(field_data.当前时间 <= ped_time)&(field_data.当前时间 > starttime)].is_ped) # 该场区该时间段应该有多少检出PED
    prob_list = list(field_data.loc[(field_data.当前时间 <= ped_time)&(field_data.当前时间 > starttime)].预测PED概率 > 0.5) #此段时间是否有预测成功
    
    TT += sum(TT_list)
    TF_TT += sum(prob_list)
    TT_FT += sum(is_ped_list)

In [395]:
TT

165.0

In [396]:
TF_TT

257

In [397]:
TT / TF_TT #准确

0.642023346303502

In [399]:
TT_FT

188.0

In [400]:
TT / TT_FT #召回

0.8776595744680851